In [67]:
instance = instanceId = InstanceId
apiUrl = ApiUrl
schoolYear = SchoolYear
DistrictId = DistrictID = districtId = districtID
apiLimit = batchLimit

prepareEdFiMetaData = prepareEdFiMetadata

### URL Initializations

In [68]:
%run OEA/modules/Ed-Fi/v0.6/src/utilities/edfi_v0_6_fetch_urls

In [69]:
instance_id = instanceId
school_year = schoolYear
api_url = apiUrl

edfi_api_manager = EdFiApiManager(api_url, instance_id, school_year)
edfi_api_manager.update_urls()
edfi_api_manager.set_other_metadata()

dependenciesUrl = edfi_api_manager.dependencies_url
openApiMetadataUrl = edfi_api_manager.openapi_metadata_url
dataManagementUrl = edfi_api_manager.data_management_url
authUrl = edfi_api_manager.auth_url

changeQueriesUrl = edfi_api_manager.get_referenced_url('Change-Queries')
changeQueriesUrl = changeQueriesUrl[:-13].replace('/metadata/', '/')
swagger_url = swaggerUrl = edfi_api_manager.get_referenced_url('Resources')

apiVersion = edfi_api_manager.api_version
apiVersion = apiVersion[1:] if apiVersion.startswith('v') else apiVersion

### OEA Initializations

In [70]:
%run OEA/modules/Ed-Fi/v0.6/src/utilities/edfi_v0_6_edfi_py

In [71]:
oea = EdFiOEAChild()   
oea.set_workspace(workspace)

In [72]:
if parameterized == True:
    edfiEntitiesPath = f'stage1/Transactional/{moduleName}/{apiVersion}/DistrictId={districtPath}/SchoolYear={schoolYearPath}/etl_entities/current_run_data'
    
    edfiEntities, _ = edfi.listSpecifiedEntities(edfiEntitiesPath)
    entitiesToFetch = edfiEntities
else:
    entitiesToFetch = 'All'

### Main Code

In [ ]:
from datetime import datetime
import math
source_path = f'stage1/Transactional/Ed-Fi/{apiVersion}/DistrictId={districtId}/SchoolYear={schoolYear}/metadata-assets/frequency_etl.csv'  
destination_path = source_path #f'stage1/Transactional/Ed-Fi/{apiVersion}/DistrictId={districtId}/SchoolYear={schoolYear}/metadata-assets/frequency_based_etl.csv'  
logs_path = f"stage1/Transactional/Ed-Fi/{apiVersion}/DistrictId={districtId}/SchoolYear={schoolYear}/metadata-assets/_frequency_etl_logs/run_logs_{datetime.today().strftime('%Y-%m-%d')}.csv"

In [74]:
exception = None
entitiesToFetch = "All"
from datetime import datetime
import math

try:
    edfiAPIClient = EdFiClient(workspace = workspace, 
                                    kvName = kvName, #NOTE: Default to None 
                                    moduleName = moduleName, 
                                    authUrl = authUrl, 
                                    dataManagementUrl = dataManagementUrl, 
                                    changeQueriesUrl = changeQueriesUrl, 
                                    dependenciesUrl = dependenciesUrl, 
                                    apiVersion = apiVersion, 
                                    batchLimit = batchLimit, 
                                    minChangeVer = minChangeVer, 
                                    maxChangeVer = maxChangeVer,
                                    schoolYear = schoolYear,
                                    districtId = districtId,
                                    clientId = client_id,
                                    clientSecret = client_secret_id)
except Exception as e:
    exception = e
    logger.error(exception)

    raise(exception)

In [ ]:
processor = EntityFrequencyProcessor(oea = oea, 
                                     filepath = source_path, 
                                     highFrequentDelta = highFrequentDelta,#0.005, 
                                     moderateFrequentDelta = moderateFrequentDelta, #5, 
                                     lowFrequentDelta = lowFrequentDelta, #10, 
                                     descriptorsDelta = descriptorsDelta) #360)

In [ ]:
processor.load_lookup_df()
processor.write_lookup_df(logs_path)
entities_to_etl, _ = processor.return_entities_to_etl()

if len(entities_to_etl) != 0:
    edfiAPIClient.landEntities(entities_to_etl, False)
else:
    logger.info("No entities to be fetched has been specified")

In [75]:
try:
    edfiAPIClient.landEntity(resource = '/ed-fi/schoolYearTypes',
                          minChangeVersion = None,
                          maxChangeVersion = None, 
                          debugMode = True)
except Exception as schoolYearTypesError:
    logger.exception('Landing of schoolYearTypes failed')
    logger.exception(schoolYearTypesError)

    raise(schoolYearTypesError)

In [77]:
# %run Dev/Ed-Fi/v0.6/dev_v0_6_EdFi_Ingest

In [24]:
# %run Dev/Ed-Fi/v0.4/dev_v0_4_EdFi_Refine